In [8]:
import requests
from bs4 import BeautifulSoup

In [9]:
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&sc=13103&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=3'

In [10]:
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="js-bukkenList")
elements = results.find_all("div", class_="cassetteitem")

In [ ]:

#print(results.prettify)

In [ ]:
elements[0]

<div class="cassetteitem">
<div class="cassetteitem-detail">
<div class="cassetteitem-detail-object">
<div class="cassetteitem_object">
<div class="cassetteitem_object-item">
<img alt="" class="js-noContextMenu js-linkImage js-scrollLazy js-adjustImg" height="0" rel="https://img01.suumo.com/front/gazo/fr/bukken/210/100351321210/100351321210_gw.jpg" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAAAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw%3D%3D" width="0">
</img></div>
</div>
</div>
<div class="cassetteitem-detail-body">
<div class="cassetteitem_content">
<div class="cassetteitem_content-label"><span class="ui-pct ui-pct--util1">賃貸マンション</span></div>
<div class="cassetteitem_content-title">Moon Island Tower</div>
<div class="cassetteitem_content-body">
<ul class="cassetteitem_detail">
<li class="cassetteitem_detail-col1">東京都中央区月島２</li>
<li class="cassetteitem_detail-col2">
<div class="cassetteitem_detail-text">東京メトロ有楽町線/月島駅 歩1分</div>
<div class="cassetteitem_detail-text">都営大江戸線/勝どき駅 歩11分</div

In [ ]:
## construction
elements[0].find("li",class_="cassetteitem_detail-col3").text.strip().split('\n')

['築22年', '地下2地上38階建']

In [ ]:
## transportation
stations = elements[0].find_all("div",class_ = "cassetteitem_detail-text")
station_detail = []
for station in stations:
  station_detail.append(station.text.strip())
print(station_detail)


In [ ]:
# apartment name
elements[0].find("div","cassetteitem_content-title").text

'Moon Island Tower'

In [ ]:
## apartment address
elements[0].find("li","cassetteitem_detail-col1").text


'東京都中央区月島２'

In [ ]:
## floor
elements[0].find_all("tr","js-cassette_link")[0].find_all('td')[2].text.strip()

'10階'

In [ ]:
## rent
elements[0].find_all("tr","js-cassette_link")[0].find("span",class_ = 'cassetteitem_price cassetteitem_price--rent').text.split('万円')[0]

'16.9'

In [ ]:
## layout
elements[0].find_all("tr","js-cassette_link")[0].find("span",class_ = 'cassetteitem_madori').text

'1LDK'

In [ ]:
elements[0].find_all("tr","js-cassette_link")[0].find("span",class_ = 'cassetteitem_menseki').text

'47.57m2'

In [11]:
import pandas as pd

In [23]:
def scrape(url):
  df = pd.DataFrame()
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  results = soup.find(id="js-bukkenList")
  elements = results.find_all("div", class_="cassetteitem")
  for ele in elements:
    mansion_name = ele.find("div","cassetteitem_content-title").text
    mansion_address = ele.find("li","cassetteitem_detail-col1").text
    mansion_detail = ele.find("li",class_="cassetteitem_detail-col3").text.strip().split('\n')
    stations = ele.find_all("div",class_ = "cassetteitem_detail-text")
    station_detail = []
    for station in stations:
      station_detail.append(station.text.strip())
    flats = ele.find_all("tr","js-cassette_link")
    for flat in flats:
      flat_dict = {'flat_name':mansion_name,'flat_address':mansion_address,'flat_detail':mansion_detail,'station':station_detail}
      floor = flat.find_all('td')[2].text.strip()
      rent = flat.find("span",class_ = 'cassetteitem_price cassetteitem_price--rent').text.split('万円')[0]
      size = flat.find("span",class_ = 'cassetteitem_menseki').text
      flat_dict['size']=size
      flat_dict['rent']=rent
      flat_dict['floor'] = floor
      one_row = pd.DataFrame({k: [v] for k, v in flat_dict.items()})
      df = pd.concat([df,one_row])
  return df



In [12]:
url_prefix = f'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&sc=13103&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page='
url = url_prefix+str(1)

In [17]:
url_bucket = [url_prefix+str(x) for x in range(1,4)]

In [32]:

df_all = pd.DataFrame()
for url_ in url_bucket:
  df = scrape(url_)
  df_all = pd.concat([df_all,df])
  print(df_all.shape)

(175, 7)
(473, 7)
(576, 7)


In [28]:
url_bucket

['https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&sc=13103&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=1',
 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&sc=13103&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=2',
 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&sc=13103&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=3']

In [29]:
url

'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&sc=13103&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=1'

In [33]:
df_all

,flat_name,flat_address,flat_detail,station,size,rent,floor
0,赤坂4丁目戸建,東京都港区赤坂４,"[築18年, 3階建]","[東京メトロ千代田線/赤坂駅 歩6分, 東京メトロ銀座線/赤坂見附駅 歩8分, 東京メトロ銀...",91.87m2,57,-
0,赤坂4丁目戸建,東京都港区赤坂４,"[築18年, 3階建]","[東京メトロ千代田線/赤坂駅 歩6分, 東京メトロ銀座線/赤坂見附駅 歩8分, 東京メトロ銀...",82.7m2,57,2階
0,Brillia日本橋三越前 ブリリア日本橋三越前,東京都中央区日本橋本町３,"[築8年, 12階建]","[東京メトロ日比谷線/小伝馬町駅 歩3分, 東京メトロ銀座線/三越前駅 歩9分, ＪＲ山手線...",58.78m2,28.5,10階
0,Brillia日本橋三越前 ブリリア日本橋三越前,東京都中央区日本橋本町３,"[築8年, 12階建]","[東京メトロ日比谷線/小伝馬町駅 歩3分, 東京メトロ銀座線/三越前駅 歩9分, ＪＲ山手線...",72.75m2,37.5,3階
0,トルナーレ日本橋浜町,東京都中央区日本橋浜町３,"[築19年, 地下2地上47階建]","[東京メトロ半蔵門線/水天宮前駅 歩5分, 東京メトロ日比谷線/人形町駅 歩7分, 都営新宿...",57.83m2,23.5,9階
...,...,...,...,...,...,...,...
0,ザ・パークハビオ麻布十番,東京都港区六本木５,"[築3年, 地下1地上14階建]","[東京メトロ南北線/麻布十番駅 歩4分, 東京メトロ南北線/六本木一丁目駅 歩9分, 東京メ...",67.26m2,41.5,5階
0,ザ・パークハビオ麻布十番,東京都港区六本木５,"[築3年, 地下1地上14階建]","[東京メトロ南北線/麻布十番駅 歩4分, 東京メトロ南北線/六本木一丁目駅 歩9分, 東京メ...",67.26m2,41.5,5階
0,ザ・パークハビオ麻布十番,東京都港区六本木５,"[築3年, 地下1地上14階建]","[東京メトロ南北線/麻布十番駅 歩4分, 東京メトロ南北線/六本木一丁目駅 歩9分, 東京メ...",67.26m2,41.9,5階
0,ザ・パークハビオ麻布十番,東京都港区六本木５,"[築3年, 地下1地上14階建]","[東京メトロ南北線/麻布十番駅 歩4分, 東京メトロ南北線/六本木一丁目駅 歩9分, 東京メ...",67.26m2,45,6階
